In [3]:
import pandas as pd
import numpy as np
import cleaning_functions

In [4]:
def extract_domain(tag):
    # Count the number of occurrences of 'Domain:'
    domain_count = tag.count('Domain:')
    
    # If there are multiple 'Domain:', mark as "DUP"
    if domain_count > 1:
        return "DUP"
    # Otherwise, extract the Domain
    else:
        match = pd.Series(tag).str.extract(r'Domain: ([\w\s]+),')
        return match.iloc[0, 0]
    
def extract_type(tag):
    print(f"Processing tag: {tag}")  # Debugging print
    type_count = tag.count('Type:')
    
    if type_count > 1:
        print("Multiple 'Type:' found.")  # Debugging print
        return "DUP"
    elif type_count == 1:
        match = pd.Series(tag).str.extract(r'Type: (.+)$')
        print(f"Extracted match: {match}")  # Debugging print
        if not match.empty and match.iloc[0, 0] is not None:
            return match.iloc[0, 0]
        else:
            print("No match found.")  # Debugging print
            return None
    else:
        print("'Type:' not found.")  # Debugging print
        return None

In [1]:
# GLOBAL VARIABLES
drop_columns = ["Task", "Billable", "Amount ()"]
raw_read_csv = '../data/S2023.csv'
duplicate_types_csv = '../data/double_entry_type.csv'
duplicate_domain_csv = '../data/double_entry_domain.csv'
resave_dups = True

In [11]:
sem = raw_read_csv.split('/')[-1].split('.')[0]

In [5]:
# import data file from semester

time_track_raw_df = pd.read_csv(raw_read_csv)

print(time_track_raw_df.dtypes)
print(time_track_raw_df.count())

Client          object
Project         object
Task           float64
Description     object
Billable        object
Start date      object
Start time      object
End date        object
End time        object
Duration        object
Tags            object
Amount ()      float64
dtype: object
Client         339
Project        339
Task             0
Description    339
Billable       339
Start date     339
Start time     339
End date       339
End time       339
Duration       339
Tags           339
Amount ()        0
dtype: int64


In [6]:
# remove columns
time_track_dropped = time_track_raw_df.drop(columns=drop_columns, inplace=False)

time_track_dropped.head()

,Client,Project,Description,Start date,Start time,End date,End time,Duration,Tags
0,S2023,Chemistry 1,Intro Survey,2023-01-10,14:00:47,2023-01-10,14:24:46,00:23:59,Domain: School
1,S2023,Computer Organization,WS1,2023-01-11,09:09:11,2023-01-11,09:23:06,00:13:55,Domain: School
2,S2023,Database Management,Lab 0,2023-01-11,13:19:45,2023-01-11,13:45:31,00:25:46,Domain: School
3,S2023,Database Management,psql configuration,2023-01-11,13:45:45,2023-01-11,14:02:40,00:16:55,Domain: School
4,S2023,ExpModernWorld,Assignment and Reading Imports,2023-01-12,13:00:00,2023-01-12,13:30:00,00:30:00,Domain: School


In [7]:
# typecast column duration to timedelta
time_track_dropped["Duration"] = pd.to_timedelta(time_track_dropped["Duration"])

# typecast column date to datetime
time_track_dropped["Start date"] = pd.to_datetime(time_track_dropped["Start date"])
time_track_dropped["End date"] = pd.to_datetime(time_track_dropped["End date"])

# typecast Tags
time_track_dropped['Tags'] = time_track_dropped['Tags'].astype(str)

print(time_track_dropped.dtypes)

time_track_typed = time_track_dropped

Client                  object
Project                 object
Description             object
Start date      datetime64[ns]
Start time              object
End date        datetime64[ns]
End time                object
Duration       timedelta64[ns]
Tags                    object
dtype: object


In [8]:
# separate Domain and Type

# Extracting Domain and Type into separate columns
time_track_typed['Type'] = time_track_typed['Tags'].apply(extract_type)
time_track_typed['Domain'] = time_track_typed['Tags'].apply(extract_domain)

time_track_typed.head()

Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.
Processing tag: Domain: School
'Type:' not found.


,Client,Project,Description,Start date,Start time,End date,End time,Duration,Tags,Type,Domain
0,S2023,Chemistry 1,Intro Survey,2023-01-10,14:00:47,2023-01-10,14:24:46,0 days 00:23:59,Domain: School,None,NaN
1,S2023,Computer Organization,WS1,2023-01-11,09:09:11,2023-01-11,09:23:06,0 days 00:13:55,Domain: School,None,NaN
2,S2023,Database Management,Lab 0,2023-01-11,13:19:45,2023-01-11,13:45:31,0 days 00:25:46,Domain: School,None,NaN
3,S2023,Database Management,psql configuration,2023-01-11,13:45:45,2023-01-11,14:02:40,0 days 00:16:55,Domain: School,None,NaN
4,S2023,ExpModernWorld,Assignment and Reading Imports,2023-01-12,13:00:00,2023-01-12,13:30:00,0 days 00:30:00,Domain: School,None,NaN


In [9]:
# look for double-entry tags and remove them

if (time_track_typed.loc[time_track_typed['Domain'] == "DUP"].size > 0): 
    print("WARNING: Double entry in Domain")
    print(f"\tEntries will be removed and placed into {duplicate_domain_csv}")
    print("\tThe above .csv will be re-included at the beginning of the analysis script")

    
    # filter out entries where Domain == DUP and put in its own csv
    if (resave_dups):
        time_track_duplicated = time_track_typed.loc[time_track_typed['Domain'] == "DUP"]
        time_track_duplicated.to_csv(duplicate_domain_csv)


    # remove entries where Domain == DUP
    time_track_typed = time_track_typed.loc[time_track_typed['Domain'] != "DUP"]

else: 
    print("No double entries in Domain")
    # save empty dataframe to csv
    time_track_duplicated = pd.DataFrame(columns=time_track_typed.columns)
    time_track_duplicated.to_csv(duplicate_domain_csv)

if (time_track_typed.loc[time_track_typed['Type'] == "DUP"].size > 0): 
    print("Double entry in Type")
    print(f"\tEntries will be removed and placed into {duplicate_types_csv}")
    print("\tThe above .csv will be re-included at the beginning of the analysis script")
    
    # filter out entries where Type == DUP and put in its own csv
    if (resave_dups):
        time_track_duplicated = time_track_typed.loc[time_track_typed['Type'] == "DUP"]
        time_track_duplicated.to_csv(duplicate_types_csv)

    # remove entries where Type == DUP
    time_track_typed = time_track_typed.loc[time_track_typed['Type'] != "DUP"]
    
else: 
    print("No double entries in Entry")
    # save empty dataframe with columns to csv
    time_track_duplicated = pd.DataFrame(columns=time_track_typed.columns)
    time_track_duplicated.to_csv(duplicate_types_csv)



In [12]:
# save cleaned data to csv
time_track_typed.to_csv(f'../data/{sem}_cleaned.csv')